## Negation 1

In [48]:
data = [
         {'lang': 'en',
        'context': 'Monica and Lisa are not doctors, but Mary and Susan are.',
         'question': 'Who are doctors?',
         'answer': 'Mary and Susan.'},
        {'lang': 'de',
        'context': 'Anna und Ulrike sind keine Ingenieurinnen, aber Robert und Hans sind es.',
         'question': 'Wer sind Ingenieure?',
         'answer': 'Robert und Hans.'},
         {'lang': 'fi',
        'context': 'Hilppa ja Henriikka eivät ole poliiseja mutta Kaija ja Juuli ovat.',
         'question': 'Ketkä ovat poliisit?',
         'answer': 'Kaija ja Juuli.'},
         # normal
         {'lang': 'ja',
        'context': '陽向と陽翔は弁護士ではないですが、誠と清はそうです。',
         'question': '何方が弁護士ですか。',
         'answer': '誠と清です。'},
         # familiar
         {'lang': 'ja',
        'context': '真由美と敬子は作家じゃないけど、明梨と優花はそうだ。',
         'question': '誰が作家？',
         'answer': '明梨と優花。'},
         # formal
         {'lang': 'ja',
        'context': '明梨様と優花様は大工ではございませんが、誠様と清様はでございます。',
         'question': '誰が大工でございますか。',
         'answer': '誠様と清様でございます。'},]


In [163]:
patterns = {
    'en': { # {N1} (is/are) {NEG} {P2}, but {P1} is/are.
           'negation1': r".*"
                        r"C\:\s(?P<N1>.+?)\s(?:is|are)\snot\s((?:a|an)\s)?(?P<P2>.+?),\sbut\s(?P<P1>.+?)\s(?:is|are)\."
                        r".*",
                        
            # {N1} (is/are) a {N2}, {P1} are {P2}.
           'negation2': r".*"
                        r"C\:\s(?P<N1>.+?)\s(?:is|are)\s((?:a|an)\s)?(?P<N2>.+?),\s(?P<P1>.+?)\s(?:is|are)\s((?:a|an)\s)?(?P<P2>.+?)\."
                        r".*",
            # C: There are {N1} {N2} and {N3} {N4} {P3}. 
            # Q: How many {P2} in total are {P3}? 
            # A: {P1}.
           'numerals1': r".*"
                        r"C\:\sThere\s(?:is|are)\s(?P<N1A>.+?)\s(?P<N2A>.+?)\sand\s(?P<N1B>.+?)\s(?P<N2B>.+?)\s(?P<P3>.+?)\.\s.*"
                        r"Q\:\sHow\smany\s(?P<P2>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",

            # C: There are {N1A} {P2} and {N1B} {N2} {P3}. Kirk ate {N1C} pineapples. 
            # Q: How many pineapples are on the table? 
            # A: {P1}.
           'numerals2': r".*"
                        r"C\:\sThere\s(?:is|are)\s(?P<N1A>.+?)\s(?P<P2>.+?)\sand\s(?P<N1B>.+?)\s(?P<N2>.+?)\s(?P<P3>.+?)\.\s.*ate\s(?P<N1C>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",

            # C: The {N2} is {N1} and the {P2} are {P1}. 
            # Q: Where are the phones? 
            # A: Next to the shelf.
           'spatial1': r".*"
                       r"C\:\sThe\s(?P<N2>.+?)\s(?:is|are)\s(?P<N1>.+?)\sand\sthe\s(?P<P2>.+?)\s(?:is|are)\s(?P<P1>.+?)\."
                       r".*",
              # C: The {P2} and the {N2} are {P1}. Kirk puts the pens {N1}. 
              # Q: Where is the book? 
              # A: Behind the table.
           'spatial2': r".*"
                       r"C\:\sThe\s(?P<P2>.+?)\sand\sthe\s(?P<N2>.+?)\sare\s(?P<P1>.+?)\.\s\w+\sputs\sthe\s\w+\s(?P<N1>.+?)\."
                       r".*",
           'temporal1': r"",
           'temporal2': r"",
           'comparative1': r"",
           'comparative2': r"",
           },
    'de': { # {N1} (ist/sind) {NEG} {P2}, aber {P1} ist/sind es.'
           'negation1': r".*"
                        r"C\:\s(?P<N1>.+?)\s(?:ist|sind)\skeine\s(?P<P2>.+?),\saber\s(?P<P1>.+?)\s(?:ist|sind)\ses\."
                        r".*",

            # {N1} ist/sind ein {N2} und {P1} sind {P2}.
           'negation2': r".*"
                        r"C\:\s(?P<N1>.+?)\s(?:ist|sind)\s(eine?\s)?(?P<N2>.+?)\sund\s(?P<P1>.+?)\s(?:ist|sind)\s(eine?\s)?(?P<P2>.+?)\."
                        r".*",

            # C: Es gibt {N1} {N2} und {N3} {N4} {P3}. 
            # Q: Wie viele {P2} liegen insgesamt {P3}? 
            # A: {P1}.
           'numerals1': r".*"
                        r"C\:\sEs\sgibt\s(?P<N1A>.+?)\s(?P<N2A>.+?)\sund\s(?P<N1B>.+?)\s(?P<N2B>.+?)\s(?P<P3>.+?)\.\s.*"
                        r"Q\:\sWie\sviele\s(?P<P2>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",
            # C: Es gibt {N1A} {P2} und {N1B} {N2} {P3}. Katja aß {N1C} Wassermelone. 
            # Q: Wie viele Wassermelonen gibt es auf dem Tisch? 
            # A: {P1}.
           'numerals2': r".*"
                        r"C\:\sEs\sgibt\s(?P<N1A>.+?)\s(?P<P2>.+?)\sund\s(?P<N1B>.+?)\s(?P<N2>.+?)\s(?P<P3>.+?)\.\s.*aß\s(?P<N1C>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",
            # C: Der {N2} liegt {N1} und die {P2} liegen {P1}. 
            # Q: Wo sind die Bücher? 
            # A: Vor dem Regal.
           'spatial1': r".*"
                       r"C\:\s(?:Der|Die|Das)\s(?P<N2>.+?)\s(?:liegt|liegen)\s(?P<N1>.+?)\sund\s(?:der|die|das)\s(?P<P2>.+?)\s(?:liegt|liegen)\s(?P<P1>.+?)\."
                       r".*",
              # C: Das {P2} und die {N2} liegen {P1}. Judith legt die Papiere {N1}. 
              # Q: Wo ist das Buch? 
              # A: Vor dem Sofa.
           'spatial2': r".*"
                       r"C\:\s(?:Der|Die|Das)\s(?P<P2>.+?)\sund\s(?:der|die|das)\s(?P<N2>.+?)\s(?:liegt|liegen)\s(?P<P1>.+?)\.\s\w+\slegt\s(?:der|die|das)\s\w+\s(?P<N1>.+?)\."
                       r".*",
           'temporal1': r"",
           'temporal2': r"",
           'comparative1': r"",
           'comparative2': r"",
           },
           
    'fi': { # {N1} {NEG} ole {P2} mutta {P1} on/ovat.
           'negation1': r".*"
                        r"C\:\s(?P<N1>.+?)\s(?:ei|eivät)\sole\s(?P<P2>.+?)\smutta\s(?P<P1>.+?)\s(?:on|ovat)\."
                        r".*",

           # {N1} on {N2}, {P1} ovat {P2}. 
           'negation2': r".*"
                        r"C\:\s(?P<N1>.+?)\s(?:on|ovat)(?P<N2>.+?),\s(?P<P1>.+?)\s(?:on|ovat)(?P<P2>.+?)"
                        r".*",
              # C: {P3} on {N1} {N2} ja {N3} {N4}. 
              # Q: Kuinka monta {P2} on {P3} yhteensä? 
              # A: {P1}.
           'numerals1': r".*"
                        r"C\:\s(?P<P3>.+?)\son\s(?P<N1A>.+?)\s(?P<N2A>.+?)\sja\s(?P<N1B>.+?)\s(?P<N2B>.+?)\.\s.*"
                        r"Q\:\sKuinka\smonta\s(?P<P2>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",
              # C: {P3} on {N1A} {P2} ja {N1B} {N2}. Ylvä söi {N1C} vesimelonia. 
              # Q: Kuinka monta vesimelonia on pöydällä? '
              # A: {P1}.        
           'numerals2': r".*"
                        r"C\:\s(?P<P3>.+?)\son\s(?P<N1A>.+?)\s(?P<P2>.+?)\sja\s(?P<N1B>.+?)\s(?P<N2>.+?)\.\s.*söi\s(?P<N1C>.+?)\s.*"
                        r"A:\s(?P<P1>.+?)\."
                        r".*",
              # C: {N2} on {N1} ja {P2} {P1}. 
              # Q: Missä ovat paperit? 
              # A: Tuolin edessä.
           'spatial1': r".*"
                       r"C\:\s(?P<N2>.+?)\s(?:on|ovat)\s(?P<N1>.+?)\sja\s(?P<P2>.+?)\s(?P<P1>.+?)\."
                       r".*",
              # C: {P2} ja {N2} ovat {P1}. Okko laittaa paperit {N1}. 
              # Q: Missä on kynä? 
              # A: Hyllyn päällä.
           'spatial2': r".*"
                       r"C\:\s(?P<P2>.+?)\sja\s(?P<N2>.+?)\s(?:on|ovat)\s(?P<P1>.+?)\.\s\w+\slaittaa\s\w+\s(?P<N1>.+?)\."
                       r".*",
           'temporal1': r"",
           'temporal2': r"",
           'comparative1': r"",
           'comparative2': r"",
           },
    'ja': {# {N1}は{P2}(じゃないけど/ではないですが/ございませんが)、{P1}はそうです。
           'negation1': r".*"
                        r"C\:\s(?P<N1>.+?)は(?P<P2>.+?)(?:じゃないけど|ではないですが|ございませんが)、(?P<P1>.+?)(?:はそうです|はそうだ|はでございます)。"
                        r".*",

           # {N1}は{N2}、{P1}は{P2}だ。
           'negation2': r".*"
                        r"C\:\s(?P<N1>.+?)は(?P<N2>.+?)、(?P<P1>.+?)は(?P<P2>.+?)(?:だ|です|でございます)。"
                        r".*",
       
              # C: {P3}に{N1}の{N2}と{N3}の{N4}がある。
              # Q: {P3}にはいくつの{P2}がある？
              # A: {P1}。
           'numerals1': r".*"
                        r"C\:\s(?P<P3>.+?)に(?P<N1A>.+?)の(?P<N2A>.+?)と(?P<N1B>.+?)の(?P<N2B>.+?)が(?:ある|あります|ございます)。.*"
                        r"Q\:\s.*にはいくつの(?P<P2>.+?)が(?:ある|あります|ございます).*"
                        r"A:\s(?P<P1>.+?)(?:です|ございます)?。"
                        r".*",

              # C: {P3}に{N1A}の{P2}と{N1B}の{N2}がございます。富美子はりんごを二個召し上がりました。
              # Q: テーブルの上にりんごはいくつございますか。
              # A: {P1}ございます。        
           'numerals2': r".*"
                        r"C\:\s(?P<P3>.+?)に(?P<N1A>.+?)の(?P<P2>.+?)と(?P<N1B>.+?)の(?P<N2>.+?)が(?:ある|あります|ございます)。.*を(?P<N1C>.+?)(?:食べた|食べました|召し上がりました)。.*"
                        r"A:\s(?P<P1>.+?)(?:です|ございます)?。"
                        r".*",
            # C: {N2}は{N1}、{P2}は{P1}にございます。
            # Q: 紙はどこにございますかか。
            # A: 机の上にございます。
           'spatial1': r".*"
                       r"C\:\s(?P<N2>.+?)は(?P<N1>.+?)、(?P<P2>.+?)は(?P<P1>.+?)に(?:ある|あります|ございます)。"
                       r".*",
            # C: {P2}と{N2}は{P1}にございます。富美子様は電話を{N1}に置かれました。
            # Q: 本はどこにございますか。
            # A: ソファの下にございます。
           'spatial2': r".*"
                       r"C\:\s(?P<P2>.+?)と(?P<N2>.+?)は(?P<P1>.+?)(?:だ|にあります|にございます)。.*を(?P<N1>.+?)に(?:置いた|置きます|置かれました)。"
                       r".*",
           'temporal1': r"",
           'temporal2': r"",
           'comparative1': r"",
           'comparative2': r"",
           },
}

conj = {"en": "and",
        "de": "und",
        "fi": "ja",
        "ja": "と"}

neg = {"en": ("not", "no"),
        "de": ("keine", "kein"),
        "fi": ("ei", "eivät"),
        "ja": ("じゃない", "ではない", 
               "じゃありません", "ではありません", 
               "ございません")}

In [167]:
import re
"""
### NEGATION1
# EN
# {NP} is/are {NEG} {J}, but {P} is/are.
# Who are {J}?
# {P}.

# DE
# {NP} ist/sind {NEG} {J}, aber {P} ist/sind es.'
# Wer ist/sind {J}?
# {P}.

# FI
# {NP} {NEG} ole {J} mutta {P} on/ovat.
# Kuka/Ketkä on/ovat {J}
# {P}.

# JA_FAM
# {NP}は{J}{NEG}ですが、{P}はそうです。
# 何方が{J}ですか。
# {P}です。
"""
"""
### NEGATION2

# EN
# {N1} (is/are) a {N2}, {P1} are {P2}. 
# Who is/are not a? {N2}? 
# {P1}.           

# DE
# {N1} ist/sind ein {N2} und {P1} sind {P2}.
# Wer sind keine {N2}?
# {P1}.

# FI
# {N1} on {N2}, {P1} ovat {P2}. 
# Ketkä eivät ole {N2}? 
# {P1}.

# JA
# {N1}は{N2}、{P1}は{P2}だ。
# {N2}じゃないのは誰？
# {P1}。


# EXP1: P1 in answer
# EXP2: P1 in answer, NEG not in answer
# EXP3: P1 in answer, N1 not in answer
# EXP4: P1 in answer, N2 not in answer
# EXP5: P1 in answer, NEG not in answer, N2 not in answer
# EXP6: P1 in answer, NEG not in answer, N1 not in answer
# EXP7: P1 in answer, N1 not in answer, N2 not in answer
# EXP8: P1 in answer, N1 not in answer, N2 not in answer, NEG not in answer

# EXP1: P1 in answer, P2 in answer,
# EXP2: P1 in answer, P2 in answer, NEG not in answer
# EXP3: P1 in answer, P2 in answer, N1 not in answer
# EXP4: P1 in answer, P2 in answer, N2 not in answer
# EXP5: P1 in answer, P2 in answer, NEG not in answer, N2 not in answer
# EXP6: P1 in answer, P2 in answer, NEG not in answer, N1 not in answer
# EXP7: P1 in answer, P2 in answer, N1 not in answer, N2 not in answer
# EXP8: P1 in answer, P2 in answer, N1 not in answer, N2 not in answer, NEG not in answer
"""
"""
### NUMERALS1
# EN
# There are {N1} {N2} and {N3} {N4} {P3}. 
# How many {P2} in total are {P3}? 
# {P1}.

# DE
# C: Es gibt {N1} {N2} und {N3} {N4} {P3}. 
# Q: Wie viele {P2} liegen insgesamt {P3}? 
# A: {P1}.

# FI
# C: Pöydällä on {N1} {N2} ja {N3} {N4}. 
# Q: Kuinka monta {P2} on {P3} yhteensä? 
# A: {P1}.

# JA
# C: {P3}に{N2}の{N1}と{N4}の{N3}がある。
# Q: {P3}にはいくつの{P2}がある？
# A: {P1}。
"""
"""
### NUMERALS2
# EN
# C: There are {N1A} {P2} and {N1B} {N2} {P3}. Kirk ate {N1C} pineapples. 
# Q: How many pineapples are on the table? 
# A: {P1}.

# DE
# C: Es gibt {N1A} {P2} und {N1B} {N2} {P3}. Katja aß {N1C} Wassermelone. 
# Q: Wie viele Wassermelonen gibt es auf dem Tisch? 
# A: {P1}.

# FI
# C: {P3} on {N1A} {P2} ja {N1B} {N2}. Ylvä söi {N1C} vesimelonia. 
# Q: Kuinka monta vesimelonia on pöydällä? '
# A: {P1}.

# JA
# C: {P3}に{N1A}の{P2}と{N1B}の{N2}がございます。富美子はりんごを二個召し上がりました。
# Q: テーブルの上にりんごはいくつございますか。
# A: {P1}ございます。
"""
"""
### SPATIAL1
# EN
# C: The {N2} is {N1} and the {P2} are {P1}. 
# Q: Where are the phones? 
# A: Next to the shelf.

# DE
# C: Der {N2} liegt {N1} und die {P2} liegen {P1}. 
# Q: Wo sind die Bücher? 
# A: Vor dem Regal.

# FI
# C: {N2} on {N1} ja {P2} {P1}. 
# Q: Missä ovat paperit? 
# A: Tuolin edessä.

# JA
# C: {N2}は{N1}、{P2}は{P1}にございます。
# Q: 紙はどこにございますかか。
# A: 机の上にございます。
"""
"""
### SPATIAL2
# EN
# C: The {P2} and the {N2} are {P1}. Kirk puts the pens {N1}. 
# Q: Where is the book? 
# A: Behind the table.

# DE
# C: Das {P2} und die {N2} liegen {P1}. Judith legt die Papiere {N1}. 
# Q: Wo ist das Buch? 
# A: Vor dem Sofa.

# FI
# C: {P2} ja {N2} ovat {P1}. Okko laittaa paperit {N1}. 
# Q: Missä on kynä? 
# A: Hyllyn päällä.

# JA
# C: {P2}と{N2}はソ{P1}にございます。富美子様は電話を{N1}に置かれました。
# Q: 本はどこにございますか。
# A: ソファの下にございます。
"""
lang = 'ja'
task = 'spatial2'

# Define the input string
input_string = {"negation1": {"en": "C: Please answer the question in English. C: Susan is not an architect, but Lola is.  Q: Who is an architect?  A:",
                              "de": "C: Heike und Fredrik ist keine Managerin, aber Johann und Karin ist es.",
                              "fi": "C: Hilppa ja Henriikka eivät ole poliiseja mutta Kaija ja Juuli ovat.",
                              "ja": "C: 陽向と陽翔は弁護士ではないですが、誠と清はそうです。"},

                "negation2": {"en": "C: Steve is an engineer, Christopher and Charles are lawyers.",
                              "de": "C: Günter ist ein Kellner und Hilda und Gretel sind Wissenschaftlerinnen.",
                              "fi": "C: Jaakko on kirjanpitäjä, Säde ja Matleena ovat viljelijöitä.",
                              "ja": "C: 千代は歯科医、良子と桜は研究員だ。"},

                "numerals1": {"en": "C: There are two pineapples and six mangoes on the table. Q: How many fruits in total are on the table? A: Eight.",
                              "de": "C: Es gibt eine Mango und zwei Äpfel auf dem Tisch. Q: Wie viele Früchte liegen insgesamt auf dem Tisch? A: Drei.",
                              "fi": "C: Pöydällä on yksi kirsikka ja yksi mansikka. Q: Kuinka monta hedelmää on pöydällä yhteensä? A: Kaksi.",
                              "ja": "C: テーブルの上に七個のスイカと七個のみかんがございます。Q: テーブルの上にはいくつの果物がございますか。A: 十四個ございます。"},

                "numerals2": {"en": "C: There are six pineapples and six strawberries on the table. Kirk ate two pineapples. Q: How many pineapples are on the table? A: Four.",
                              "de": "C: Es gibt acht Wassermelonen und neun Äpfel auf dem Tisch. Katja aß eine Wassermelone. Q: Wie viele Wassermelonen gibt es auf dem Tisch? A: Sieben.",
                              "fi": "C: Pöydällä on kuusi vesimelonia ja yhdeksän banaania. Ylvä söi nolla vesimelonia. Q: Kuinka monta vesimelonia on pöydällä? A: Kuusi.",
                              "ja": "C: テーブルの上に五個のりんごと六個の桃がございます。富美子はりんごを二個召し上がりました。Q: テーブルの上にりんごはいくつございますか。A: 三個ございます。"},
                
                "spatial1": {"en": "C: The mug is behind the window and the phones are next to the shelf. Q: Where are the phones? A: Next to the shelf.",
                              "de": "C: Der Rechner liegt hinter dem Sofa und die Bücher liegen vor dem Regal. Q: Wo sind die Bücher? A: Vor dem Regal.",
                              "fi": "C: Kynä on ikkunan vieressä ja paperit tuolin edessä. Q: Missä ovat paperit? A: Tuolin edessä.",
                              "ja": "C: 紙は机の上、電話は棚の上にございます。Q: 紙はどこにございますかか。A: 机の上にございます。"},
                
                "spatial2": {"en": "C: The book and the pens are behind the table. Kirk puts the pens on the ground. Q: Where is the book? A: Behind the table.",
                              "de": "C: Das Buch und die Papiere liegen vor dem Sofa. Judith legt die Papiere auf den Boden. Q: Wo ist das Buch? A: Vor dem Sofa.",
                              "fi": "C: Kynä ja paperit ovat hyllyn päällä. Okko laittaa paperit maahan. Q: Missä on kynä? A: Hyllyn päällä.",
                              "ja": "C: 本と電話はソファの下にございます。富美子様は電話を床に置かれました。Q: 本はどこにございますか。A: ソファの下にございます。"},              
                              }

# Define the regular expression pattern
# Use re.search to find the match
match = re.search(patterns[lang][task], input_string[task][lang])
print(input_string[task][lang])
print(patterns[lang][task])
if match:
    # Print the extracted components
    
    print(match.groupdict())


C: 本と電話はソファの下にございます。富美子様は電話を床に置かれました。Q: 本はどこにございますか。A: ソファの下にございます。
.*C\:\s(?P<P2>.+?)と(?P<N2>.+?)は(?P<P1>.+?)(?:だ|にあります|にございます)。.*を(?P<N1>.+?)に(?:置いた|置きます|置かれました)。.*
{'P2': '本', 'N2': '電話', 'P1': 'ソファの下', 'N1': '床'}


### English

### German

In [21]:
de_data = data[1]

# POSITIVES
# Robert und Hans
# Hans und Robert
# Mark and Andrew are doctors
# Andrew and Mark are doctors

# NEGATIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are not doctors
# Andrew and Mark are not doctors
# Emma and Monica are doctors
# Monica and Emma are doctors

conj = ' und '

subjects = de_data['answer'].strip('.')
subjects = [subjects, conj.join(subjects.split(conj)[::-1])] if conj in subjects else [subjects]
predicate = ' '.join(de_data['question'].strip('?').split(' ')[1:])
non_subjects = de_data['context'].split(' ')
non_subjects = ' '.join(non_subjects[:3]) if conj.strip() in non_subjects[:3] else ' '.join(non_subjects[:1])
non_subjects = [non_subjects, conj.join(non_subjects.split(conj)[::-1])] if conj in non_subjects else [non_subjects]

perfect_mathes = subjects
positive_matches = [subject + ' ' + predicate for subject in subjects]
negative_matches = [subject + ' ' + predicate for subject in non_subjects]

print(perfect_mathes, positive_matches, negative_matches)





['Robert und Hans', 'Hans und Robert'] ['Robert und Hans sind Ingenieure', 'Hans und Robert sind Ingenieure'] ['Anna und Ulrike sind Ingenieure', 'Ulrike und Anna sind Ingenieure']


### Finnish

In [22]:
fi_data = data[2]

# POSITIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are doctors
# Andrew and Mark are doctors

# NEGATIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are not doctors
# Andrew and Mark are not doctors
# Emma and Monica are doctors
# Monica and Emma are doctors

conj = ' ja '

subjects = fi_data['answer'].strip('.')
subjects = [subjects, conj.join(subjects.split(conj)[::-1])] if conj in subjects else [subjects]
predicate = ' '.join(fi_data['question'].strip('?').split(' ')[1:])
non_subjects = fi_data['context'].split(' ')
non_subjects = ' '.join(non_subjects[:3]) if conj.strip() in non_subjects[:3] else ' '.join(non_subjects[:1])
non_subjects = [non_subjects, conj.join(non_subjects.split(conj)[::-1])] if conj in non_subjects else [non_subjects]

perfect_mathes = subjects
positive_matches = [subject + ' ' + predicate for subject in subjects]
negative_matches = [subject + ' ' + predicate for subject in non_subjects]

print(perfect_mathes, positive_matches, negative_matches)

['Kaija ja Juuli', 'Juuli ja Kaija'] ['Kaija ja Juuli ovat poliisit', 'Juuli ja Kaija ovat poliisit'] ['Hilppa ja Henriikka ovat poliisit', 'Henriikka ja Hilppa ovat poliisit']


### Japanese

In [40]:
ja_data = data[3]

# POSITIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are doctors
# Andrew and Mark are doctors

# NEGATIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are not doctors
# Andrew and Mark are not doctors
# Emma and Monica are doctors
# Monica and Emma are doctors

conj = 'と'
cop = 'でございます' if ja_data['answer'].endswith('でございます。') else 'です' if ja_data['answer'].endswith('です。') else ''

subjects = ja_data['answer'].strip('。')[:-len(cop)]
subjects = [subjects, conj.join(subjects.split(conj)[::-1])] if conj in subjects else [subjects]

predicate = ' '.join(ja_data['question'].rstrip('か。').split('が')[1:])

non_subjects = ja_data['context'].split('は')[0]
non_subjects = [non_subjects, conj.join(non_subjects.split(conj)[::-1])] if conj in non_subjects else [non_subjects]

perfect_mathes = [subject + cop for subject in subjects]
positive_matches = [subject + part + predicate for subject in subjects for part in ('は', 'が')]
positive_matches.extend([predicate[:-len(cop)] + 'は' + subject + cop for subject in subjects])
negative_matches = [subject + part + predicate for subject in non_subjects for part in ('は', 'が')]
negative_matches.extend([predicate[:-len(cop)] + 'は' + subject + cop for subject in non_subjects])

print(perfect_mathes)
print(positive_matches) 
print(negative_matches)

['誠と清です', '清と誠です']
['誠と清は弁護士です', '誠と清が弁護士です', '清と誠は弁護士です', '清と誠が弁護士です', '弁護士は誠と清です', '弁護士は清と誠です']
['陽向と陽翔は弁護士です', '陽向と陽翔が弁護士です', '陽翔と陽向は弁護士です', '陽翔と陽向が弁護士です', '弁護士は陽向と陽翔です', '弁護士は陽翔と陽向です']


In [38]:
ja_data = data[4]

# POSITIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are doctors
# Andrew and Mark are doctors

# NEGATIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are not doctors
# Andrew and Mark are not doctors
# Emma and Monica are doctors
# Monica and Emma are doctors

conj = 'と'
cop = 'でございます' if ja_data['answer'].endswith('でございます。') else 'です' if ja_data['answer'].endswith('です。') else ''

subjects = ja_data['answer'].strip('。 ')
subjects = [subjects, conj.join(subjects.split(conj)[::-1])] if conj in subjects else [subjects]

predicate = ' '.join(ja_data['question'].rstrip('か。？').split('が')[1:])

non_subjects = ja_data['context'].split('は')[0]
non_subjects = [non_subjects, conj.join(non_subjects.split(conj)[::-1])] if conj in non_subjects else [non_subjects]

perfect_mathes = [subject + cop for subject in subjects for cop in ('', 'だ')]
positive_matches = [subject + part + predicate for subject in subjects for part in ('は', 'が')]
positive_matches.extend([predicate + 'は' + subject + cop for subject in subjects for cop in ('', 'だ')])
negative_matches = [subject + part + predicate for subject in non_subjects for part in ('は', 'が')]
negative_matches.extend([predicate + 'は' + subject + cop for subject in non_subjects for cop in ('', 'だ')])

print(perfect_mathes)
print(positive_matches) 
print(negative_matches)

# familiar
# {'lang': 'ja',
# 'context': '真由美と敬子は作家じゃないけど、明梨と優花はそうだ。',
# 'question': '誰が作家？',
# 'answer': '明梨と優花。'},

['明梨と優花', '明梨と優花だ', '優花と明梨', '優花と明梨だ']
['明梨と優花は作家', '明梨と優花が作家', '優花と明梨は作家', '優花と明梨が作家', '作家は明梨と優花', '作家は明梨と優花だ', '作家は優花と明梨', '作家は優花と明梨だ']
['真由美と敬子は作家', '真由美と敬子が作家', '敬子と真由美は作家', '敬子と真由美が作家', '作家は真由美と敬子', '作家は真由美と敬子だ', '作家は敬子と真由美', '作家は敬子と真由美だ']


In [39]:
ja_data = data[5]

# POSITIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are doctors
# Andrew and Mark are doctors

# NEGATIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are not doctors
# Andrew and Mark are not doctors
# Emma and Monica are doctors
# Monica and Emma are doctors

conj = 'と'
cop = 'でございます' if ja_data['answer'].endswith('でございます。') else 'です' if ja_data['answer'].endswith('です。') else ''

subjects = ja_data['answer'].strip('。')[:-len(cop)]
subjects = [subjects, conj.join(subjects.split(conj)[::-1])] if conj in subjects else [subjects]

predicate = ' '.join(ja_data['question'].rstrip('か。').split('が')[1:])

non_subjects = ja_data['context'].split('は')[0]
non_subjects = [non_subjects, conj.join(non_subjects.split(conj)[::-1])] if conj in non_subjects else [non_subjects]

perfect_mathes = [subject + cop for subject in subjects]
positive_matches = [subject + part + predicate for subject in subjects for part in ('は', 'が')]
positive_matches.extend([predicate[:-len(cop)] + 'は' + subject + cop for subject in subjects])
negative_matches = [subject + part + predicate for subject in non_subjects for part in ('は', 'が')]
negative_matches.extend([predicate[:-len(cop)] + 'は' + subject + cop for subject in non_subjects])

print(perfect_mathes)
print(positive_matches) 
print(negative_matches)

['誠様と清様でございます', '清様と誠様でございます']
['誠様と清様は大工でございます', '誠様と清様が大工でございます', '清様と誠様は大工でございます', '清様と誠様が大工でございます', '大工は誠様と清様でございます', '大工は清様と誠様でございます']
['明梨様と優花様は大工でございます', '明梨様と優花様が大工でございます', '優花様と明梨様は大工でございます', '優花様と明梨様が大工でございます', '大工は明梨様と優花様でございます', '大工は優花様と明梨様でございます']


In [50]:
ja_data = data[5]

# POSITIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are doctors
# Andrew and Mark are doctors

# NEGATIVES
# Mark and Andrewです
# Monica and Emma are doctors
print(ja_data)

conj = 'と'
cop = 'でございます' if ja_data['answer'].endswith('でございます。') else 'です' if ja_data['answer'].endswith('です。') else ''

subjects = ja_data['answer'].strip('。')[:-len(cop)] if cop != '' else ja_data['answer'].strip('。 ')
subjects = [subjects, conj.join(subjects.split(conj)[::-1])] if conj in subjects else [subjects]

predicate = ' '.join(ja_data['question'].rstrip('か。？?').split('が')[1:])
predicate = predicate[:-len(cop)] if cop != '' else predicate

non_subjects = ja_data['context'].split('は')[0]
non_subjects = [non_subjects, conj.join(non_subjects.split(conj)[::-1])] if conj in non_subjects else [non_subjects]

perfect_matches = []
positive_matches = []
negative_matches = []
for subject in subjects:
    if cop == '':
        perfect_matches.append(f"{subject}")
        perfect_matches.append(f"{subject}だ")

        positive_matches.append(f"{subject}が{predicate}")
        positive_matches.append(f"{subject}が{predicate}だ")

        positive_matches.append(f"{predicate}は{subject}")
        positive_matches.append(f"{predicate}は{subject}だ")
    elif cop == 'です':
        perfect_matches.append(f"{subject}です")

        positive_matches.append(f"{subject}が{predicate}です")

        positive_matches.append(f"{predicate}は{subject}です")
    elif cop == 'でございます':
        perfect_matches.append(f"{subject}でございます")

        positive_matches.append(f"{subject}が{predicate}でございます")

        positive_matches.append(f"{predicate}は{subject}でございます")

for subject in non_subjects:
    if cop == '':
        negative_matches.append(f"{subject}が{predicate}")
        negative_matches.append(f"{subject}が{predicate}だ")

        negative_matches.append(f"{predicate}は{subject}")
        negative_matches.append(f"{predicate}は{subject}だ")
    elif cop == 'です':
        negative_matches.append(f"{subject}が{predicate}です")

        negative_matches.append(f"{predicate}は{subject}です")
    elif cop == 'でございます':
        negative_matches.append(f"{subject}が{predicate}でございます")

        negative_matches.append(f"{predicate}は{subject}でございます")

print(perfect_matches)
print(positive_matches)
print(negative_matches)


{'lang': 'ja', 'context': '明梨様と優花様は大工ではございませんが、誠様と清様はでございます。', 'question': '誰が大工でございますか。', 'answer': '誠様と清様でございます。'}
['誠様と清様でございます', '清様と誠様でございます']
['誠様と清様が大工でございます', '大工は誠様と清様でございます', '清様と誠様が大工でございます', '大工は清様と誠様でございます']
['明梨様と優花様が大工でございます', '大工は明梨様と優花様でございます', '優花様と明梨様が大工でございます', '大工は優花様と明梨様でございます']


## Negation 2

In [51]:
# C: Paul and Bob are lawyers, Ross and Joey are architects. Q: Who are not lawyers? A: Ross and Joey.

# C: Anna und Ulrike sind Kellnerinnen und Nadine und Sabine sind Polizistinnen. Q: Wer sind keine Kellner? A: Nadine und Sabine.

# C: Jaakob ja Riina ovat näyttelijöitä, Aatto ja Pekka ovat kirjanpitäjiä. Q: Ketkä eivät ole näyttelijöitä? A: Aatto ja Pekka.

# C: 悠人と進は弁護士、伶と直樹は役者だ。Q: 弁護士じゃないのは誰？A: 伶と直樹。

# C: 真由美様と敬子様は歯科医、誠様と清様は作家でございます。Q: 歯科医でないのは誰でございますか。A: 誠様と清様でございます。

# C: 明梨と優花は弁護士、結衣と智子は研究員です。Q: 弁護士でないのは誰ですか。A: 結衣と智子です。


data = [
        {'lang': 'en',
        'context': 'Paul and Bob are lawyers, Ross and Joey are architects.',
        'question': 'Who are not lawyers?',
        'answer': 'Ross and Joey.'},
        {'lang': 'de',
        'context': 'Anna und Ulrike sind Kellnerinnen und Nadine und Sabine sind Polizistinnen.',
        'question': 'Wer sind keine Kellner?',
        'answer': 'Nadine und Sabine.'},
        {'lang': 'fi',
        'context': 'Jaakob ja Riina ovat näyttelijöitä, Aatto ja Pekka ovat kirjanpitäjiä.',
        'question': 'Ketkä eivät ole näyttelijöitä?',
        'answer': 'Aatto ja Pekka.'},
         # normal
        {'lang': 'ja',
        'context': '明梨と優花は弁護士、結衣と智子は研究員です。',
        'question': '弁護士でないのは誰ですか。',
        'answer': '結衣と智子です。'},
         # familiar
        {'lang': 'ja',
        'context': '悠人と進は弁護士、伶と直樹は役者だ。',
        'question': '弁護士じゃないのは誰？',
        'answer': '伶と直樹。'},
         # formal
        {'lang': 'ja',
        'context': '真由美様と敬子様は歯科医、誠様と清様は作家でございます。',
        'question': '歯科医でないのは誰でございますか。',
        'answer': '誠様と清様でございます。'},]

In [17]:
eng_data = data[0]

# POSITIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are doctors
# Andrew and Mark are doctors

# NEGATIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are not doctors
# Andrew and Mark are not doctors
# Emma and Monica are doctors
# Monica and Emma are doctors

conj = ' and '

subjects = eng_data['answer'].strip('.')
subjects = [subjects, conj.join(subjects.split(conj)[::-1])] if conj in subjects else [subjects]
predicate = ' '.join(eng_data['question'].strip('?').split(' ')[1:])
non_subjects = eng_data['context'].split(' ')
non_subjects = ' '.join(non_subjects[:3]) if conj.strip() in non_subjects[:3] else ' '.join(non_subjects[:1])
non_subjects = [non_subjects, conj.join(non_subjects.split(conj)[::-1])] if conj in non_subjects else [non_subjects]

perfect_mathes = subjects
positive_matches = [subject + ' ' + predicate for subject in subjects]
negative_matches = [subject + ' ' + predicate for subject in non_subjects]

print(perfect_mathes, positive_matches, negative_matches)




['Ross and Joey', 'Joey and Ross'] ['Ross and Joey are not lawyers', 'Joey and Ross are not lawyers'] ['Paul and Bob are not lawyers', 'Bob and Paul are not lawyers']


In [18]:
de_data = data[1]

# POSITIVES
# Robert und Hans
# Hans und Robert
# Mark and Andrew are doctors
# Andrew and Mark are doctors

# NEGATIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are not doctors
# Andrew and Mark are not doctors
# Emma and Monica are doctors
# Monica and Emma are doctors

conj = ' und '

subjects = de_data['answer'].strip('.')
subjects = [subjects, conj.join(subjects.split(conj)[::-1])] if conj in subjects else [subjects]
predicate = ' '.join(de_data['question'].strip('?').split(' ')[1:])
non_subjects = de_data['context'].split(' ')
non_subjects = ' '.join(non_subjects[:3]) if conj.strip() in non_subjects[:3] else ' '.join(non_subjects[:1])
non_subjects = [non_subjects, conj.join(non_subjects.split(conj)[::-1])] if conj in non_subjects else [non_subjects]

perfect_mathes = subjects
positive_matches = [subject + ' ' + predicate for subject in subjects]
negative_matches = [subject + ' ' + predicate for subject in non_subjects]

print(de_data)
print(perfect_mathes, positive_matches, negative_matches)





{'lang': 'de', 'context': 'Anna und Ulrike sind Kellnerinnen und Nadine und Sabine sind Polizistinnen.', 'question': 'Wer sind keine Kellner?', 'answer': 'Nadine und Sabine.'}
['Nadine und Sabine', 'Sabine und Nadine'] ['Nadine und Sabine sind keine Kellner', 'Sabine und Nadine sind keine Kellner'] ['Anna und Ulrike sind keine Kellner', 'Ulrike und Anna sind keine Kellner']


In [19]:
fi_data = data[2]

# POSITIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are doctors
# Andrew and Mark are doctors

# NEGATIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are not doctors
# Andrew and Mark are not doctors
# Emma and Monica are doctors
# Monica and Emma are doctors

conj = ' ja '

subjects = fi_data['answer'].strip('.')
subjects = [subjects, conj.join(subjects.split(conj)[::-1])] if conj in subjects else [subjects]
predicate = ' '.join(fi_data['question'].strip('?').split(' ')[1:])
non_subjects = fi_data['context'].split(' ')
non_subjects = ' '.join(non_subjects[:3]) if conj.strip() in non_subjects[:3] else ' '.join(non_subjects[:1])
non_subjects = [non_subjects, conj.join(non_subjects.split(conj)[::-1])] if conj in non_subjects else [non_subjects]

perfect_mathes = subjects
positive_matches = [subject + ' ' + predicate for subject in subjects]
negative_matches = [subject + ' ' + predicate for subject in non_subjects]

print(perfect_mathes, positive_matches, negative_matches)

['Aatto ja Pekka', 'Pekka ja Aatto'] ['Aatto ja Pekka eivät ole näyttelijöitä', 'Pekka ja Aatto eivät ole näyttelijöitä'] ['Jaakob ja Riina eivät ole näyttelijöitä', 'Riina ja Jaakob eivät ole näyttelijöitä']


In [3]:
# C: Paul and Bob are lawyers, Ross and Joey are architects. Q: Who are not lawyers? A: Ross and Joey.

# C: Anna und Ulrike sind Kellnerinnen und Nadine und Sabine sind Polizistinnen. Q: Wer sind keine Kellner? A: Nadine und Sabine.

# C: Jaakob ja Riina ovat näyttelijöitä, Aatto ja Pekka ovat kirjanpitäjiä. Q: Ketkä eivät ole näyttelijöitä? A: Aatto ja Pekka.

# C: 悠人と進は弁護士、伶と直樹は役者だ。Q: 弁護士じゃないのは誰？A: 伶と直樹。

# C: 真由美様と敬子様は歯科医、誠様と清様は作家でございます。Q: 歯科医でないのは誰でございますか。A: 誠様と清様でございます。

# C: 明梨と優花は弁護士、結衣と智子は研究員です。Q: 弁護士でないのは誰ですか。A: 結衣と智子です。


data = [
        {'lang': 'en',
        'context': 'Paul and Bob are lawyers, Ross and Joey are architects.',
        'question': 'Who are not lawyers?',
        'answer': 'Ross and Joey.'},
        {'lang': 'de',
        'context': 'Anna und Ulrike sind Kellnerinnen und Nadine und Sabine sind Polizistinnen.',
        'question': 'Wer sind keine Kellner?',
        'answer': 'Nadine und Sabine.'},
        {'lang': 'fi',
        'context': 'Jaakob ja Riina ovat näyttelijöitä, Aatto ja Pekka ovat kirjanpitäjiä.',
        'question': 'Ketkä eivät ole näyttelijöitä?',
        'answer': 'Aatto ja Pekka.'},
         # normal
        {'lang': 'ja',
        'context': '明梨と優花は弁護士、結衣と智子は研究員です。',
        'question': '弁護士でないのは誰ですか。',
        'answer': '結衣と智子です。'},
         # familiar
        {'lang': 'ja',
        'context': '悠人と進は弁護士、伶と直樹は役者だ。',
        'question': '弁護士じゃないのは誰？',
        'answer': '伶と直樹。'},
         # formal
        {'lang': 'ja',
        'context': '真由美様と敬子様は歯科医、誠様と清様は作家でございます。',
        'question': '歯科医でないのは誰でございますか。',
        'answer': '誠様と清様でございます。'},]

ja_data = data[5]

# POSITIVES
# Mark and Andrew
# Andrew and Mark
# Mark and Andrew are doctors
# Andrew and Mark are doctors

# NEGATIVES
# Mark and Andrewです
# Monica and Emma are doctors
print(ja_data)

conj = 'と'
cop = 'でございます' if ja_data['answer'].endswith('でございます。') else 'です' if ja_data['answer'].endswith('です。') else ''

subjects = ja_data['answer'].strip('。')[:-len(cop)] if cop != '' else ja_data['answer'].strip('。 ')
subjects = [subjects, conj.join(subjects.split(conj)[::-1])] if conj in subjects else [subjects]

predicate = ja_data['question'].rstrip('か。？?').split('のは')[0].replace('じゃない', '').replace('でない', '')

non_subjects = ja_data['context'].split('は')[0]
non_subjects = [non_subjects, conj.join(non_subjects.split(conj)[::-1])] if conj in non_subjects else [non_subjects]

perfect_matches = []
positive_matches = []
negative_matches = []
for subject in subjects:
    if cop == '':
        perfect_matches.append(f"{subject}")
        perfect_matches.append(f"{subject}だ")

        positive_matches.append(f"{subject}が{predicate}じゃない")
        positive_matches.append(f"{subject}が{predicate}ではない")

        positive_matches.append(f"{predicate}じゃないのは{subject}だ")
        positive_matches.append(f"{predicate}でないのは{subject}だ")

    elif cop == 'です':
        perfect_matches.append(f"{subject}です")

        positive_matches.append(f"{subject}が{predicate}じゃありません")
        positive_matches.append(f"{subject}が{predicate}ではありません")
        positive_matches.append(f"{subject}が{predicate}じゃないです")
        positive_matches.append(f"{subject}が{predicate}ではないです")

        positive_matches.append(f"{predicate}でないのは{subject}です")
        positive_matches.append(f"{predicate}じゃないのは{subject}です")

    elif cop == 'でございます':
        perfect_matches.append(f"{subject}でございます")

        positive_matches.append(f"{subject}が{predicate}ではございません")

        positive_matches.append(f"{predicate}でないのは{subject}でございます")

for subject in non_subjects:
    if cop == '':
        negative_matches.append(f"{subject}")
        negative_matches.append(f"{subject}だ")

        negative_matches.append(f"{subject}が{predicate}じゃない")
        negative_matches.append(f"{subject}が{predicate}ではない")

        negative_matches.append(f"{predicate}じゃないのは{subject}だ")
        negative_matches.append(f"{predicate}でないのは{subject}だ")
        
    elif cop == 'です':
        negative_matches.append(f"{subject}です")

        negative_matches.append(f"{subject}が{predicate}じゃありません")
        negative_matches.append(f"{subject}が{predicate}ではありません")
        negative_matches.append(f"{subject}が{predicate}じゃないです")
        negative_matches.append(f"{subject}が{predicate}ではないです")

        negative_matches.append(f"{predicate}でないのは{subject}です")
        negative_matches.append(f"{predicate}じゃないのは{subject}です")

    elif cop == 'でございます':
        negative_matches.append(f"{subject}でございます")

        negative_matches.append(f"{subject}が{predicate}ではございません")

        negative_matches.append(f"{predicate}でないのは{subject}でございます")

print(subjects)
print(non_subjects)
print(predicate)

print(perfect_matches)
print(positive_matches)
print(negative_matches)

{'lang': 'ja', 'context': '真由美様と敬子様は歯科医、誠様と清様は作家でございます。', 'question': '歯科医でないのは誰でございますか。', 'answer': '誠様と清様でございます。'}
['誠様と清様', '清様と誠様']
['真由美様と敬子様', '敬子様と真由美様']
歯科医
['誠様と清様でございます', '清様と誠様でございます']
['誠様と清様が歯科医ではございません', '歯科医でないのは誠様と清様でございます', '清様と誠様が歯科医ではございません', '歯科医でないのは清様と誠様でございます']
['真由美様と敬子様でございます', '真由美様と敬子様が歯科医ではございません', '歯科医でないのは真由美様と敬子様でございます', '敬子様と真由美様でございます', '敬子様と真由美様が歯科医ではございません', '歯科医でないのは敬子様と真由美様でございます']


## Numerals 1

In [ ]:
# C: There are two pineapples and six mangoes on the table. Q: How many fruits in total are on the table? A: Eight.

# C: Es gibt eine Mango und zwei Äpfel auf dem Tisch. Q: Wie viele Früchte liegen insgesamt auf dem Tisch? A: Drei.

# C: Pöydällä on yksi kirsikka ja yksi mansikka. Q: Kuinka monta hedelmää on pöydällä yhteensä? A: Kaksi.

# C: テーブルの上に一個のカキと一個のマンゴーがある。Q: テーブルの上にはいくつの果物がある？A: 二個。

# C: テーブルの上に二個のあんずと三個のパパイヤがございます。Q: テーブルの上にはいくつの果物がございますか。A: 五個ございます。

# C: テーブルの上に十個のレモンと十個のキウイがあります。Q: テーブルの上にはいくつの果物がありますか。A: 二十個です。


data = [
        {'lang': 'en',
        'context': 'There are two pineapples and six mangoes on the table.',
        'question': 'How many fruits in total are on the table?',
        'answer': 'Eight.'},
        {'lang': 'de',
        'context': 'Es gibt eine Mango und zwei Äpfel auf dem Tisch.',
        'question': 'Wie viele Früchte liegen insgesamt auf dem Tisch?',
        'answer': 'Drei.'},
        {'lang': 'fi',
        'context': 'Pöydällä on yksi kirsikka ja yksi mansikka.',
        'question': 'Kuinka monta hedelmää on pöydällä yhteensä?',
        'answer': 'Kaksi.'},
         # normal
        {'lang': 'ja',
        'context': 'テーブルの上に十個のレモンと十個のキウイがあります。',
        'question': 'テーブルの上にはいくつの果物がありますか。',
        'answer': '二十個です。'},
         # familiar
        {'lang': 'ja',
        'context': 'テーブルの上に一個のカキと一個のマンゴーがある。',
        'question': 'テーブルの上にはいくつの果物がある？',
        'answer': '二個。'},
         # formal
        {'lang': 'ja',
        'context': 'テーブルの上に二個のあんずと三個のパパイヤがございます。',
        'question': 'テーブルの上にはいくつの果物がございますか。',
        'answer': '五個ございます。'},]

In [20]:
data = {'lang': 'ja',
        'context': 'テーブルの上に二個のあんずと三個のパパイヤがございます。',
        'question': 'テーブルの上にはいくつの果物がございますか。',
        'answer': '五個ございます。'}

# PERFECT:
# Eight 

# POSITIVE:
# Eight fruits

# NEGATIVE:
# pineapples
# mangoes

conj = ' ja '

number, cop = data['answer'].strip('。').split('個')
target = ['果物', 'くだもの']

non_targets = [data['context'].split('個の')[1].split('と')[0], 
               data['context'].split('個の')[-1].split('が')[0]]

print(number)
print(non_targets)

perfect_matches = data['answer'].strip('。 ')
positive_matches = f'{number}個'
negative_matches = non_targets

print(perfect_matches)
print(positive_matches)
print(negative_matches)


五
['あんず', 'パパイヤ']
五個ございます
五個
['あんず', 'パパイヤ']


## Numerals 2

In [ ]:
# C: There are eight watermelons and nine strawberries on the table. Robert ate two watermelons. Q: How many watermelons are on the table? A: Six.

# C: Es gibt acht Orangen und acht Kirschen auf dem Tisch. Katja aß zwei Orangen. Q: Wie viele Orangen gibt es auf dem Tisch? A: Sechs.

# C: Pöydällä on seitsemän banaania ja kahdeksan päärynää. Päiviö söi yhden banaanin. Q: Kuinka monta banaania on pöydällä? A: Kuusi.

# C: テーブルの上に五個の梨と五個のあんずがある。愛は梨を一個食べた。Q: テーブルの上に梨はいくつある？。A: 四個。

# C: テーブルの上に七個のパイナップルと九個のパパイヤがあります。愛はパイナップルを一個食べました。Q: テーブルの上にパイナップルはいくつありますか。A: 六個です。

# C: テーブルの上に八個のイチゴと十個のカキがございます。富美子はイチゴを二個召し上がりました。Q: テーブルの上にイチゴはいくつございますか。A: 六個ございます。

data = [
        {'lang': 'en',
        'context': 'There are eight watermelons and nine strawberries on the table. Robert ate two watermelons.',
        'question': 'How many watermelons are on the table?',
        'answer': 'Six.'},
        {'lang': 'de',
        'context': 'Es gibt acht Orangen und acht Kirschen auf dem Tisch. Katja aß zwei Orangen.',
        'question': 'Wie viele Orangen gibt es auf dem Tisch?',
        'answer': 'Sechs.'},
        {'lang': 'fi',
        'context': 'Pöydällä on seitsemän banaania ja kahdeksan päärynää. Päiviö söi yhden banaanin.',
        'question': 'Kuinka monta banaania on pöydällä?',
        'answer': 'Kuusi.'},
         # normal
        {'lang': 'ja',
        'context': 'テーブルの上に七個のパイナップルと九個のパパイヤがあります。愛はパイナップルを一個食べました。',
        'question': 'テーブルの上にパイナップルはいくつありますか。',
        'answer': '六個です。'},
         # familiar
        {'lang': 'ja',
        'context': 'テーブルの上に五個の梨と五個のあんずがある。愛は梨を一個食べた。',
        'question': 'テーブルの上に梨はいくつある？。',
        'answer': '二個。'},
         # formal
        {'lang': 'ja',
        'context': 'テーブルの上に八個のイチゴと十個のカキがございます。富美子はイチゴを二個召し上がりました。',
        'question': 'テーブルの上にイチゴはいくつございますか。',
        'answer': '六個ございます。'},]

In [30]:
data = {'lang': 'ja',
        'context': 'テーブルの上に五個の梨と五個のあんずがある。愛は梨を一個食べた。',
        'question': 'テーブルの上に梨はいくつある？。',
        'answer': '二個。'}

number = data['answer'].strip('。').split('個')[0]
target = data['question'].split('上に')[1].split('はいくつ')[0]

non_targets = [data['context'].split('個の')[2].split('が')[0]]

perfect_matches = [data['answer'].strip('。 ')]
positive_matches = [f'{number}個']
negative_matches = non_targets

print(perfect_matches)
print(positive_matches)
print(negative_matches)

['二個']
['二個']
['あんず']


## Spatial 1

In [ ]:
# C: The mug is behind the window and the phones are next to the shelf. Q: Where are the phones? A: Next to the shelf.

# C: Der Rechner liegt hinter dem Sofa und die Bücher liegen vor dem Regal. Q: Wo sind die Bücher? A: Vor dem Regal.

# C: Kynä on ikkunan vieressä ja paperit tuolin edessä. Q: Missä ovat paperit? A: Tuolin edessä.

# C: 鉛筆は机の上、電話は椅子の上にある。Q: 鉛筆はどこ？A: 机の上。

# C: 本はテーブルの上、電話は机の前にあります。Q: 本はどこにありますか。A: テーブルの上です。

# C: 紙は机の上、電話は棚の上にございます。Q: 紙はどこにございますかか。A: 机の上にございます。

data = [
        {'lang': 'en',
        'context': 'The mug is behind the window and the phones are next to the shelf.',
        'question': 'Where are the phones?',
        'answer': 'Next to the shelf.'},
        {'lang': 'de',
        'context': 'Der Rechner liegt hinter dem Sofa und die Bücher liegen vor dem Regal.',
        'question': 'Wo sind die Bücher?',
        'answer': 'Vor dem Regal.'},
        {'lang': 'fi',
        'context': 'Pöydällä on seitsemän banaania ja kahdeksan päärynää. Päiviö söi yhden banaanin.',
        'question': 'Missä ovat paperit?',
        'answer': 'Tuolin edessä.'},
         # familiar
        {'lang': 'ja',
        'context': '鉛筆は机の上、電話は椅子の上にある。',
        'question': '鉛筆はどこ？',
        'answer': '机の上。'},
         # normal
        {'lang': 'ja',
        'context': '本はテーブルの上、電話は机の前にあります。',
        'question': '本はどこにありますか。',
        'answer': 'テーブルの上です。'},
         # formal
        {'lang': 'ja',
        'context': '紙は机の上、電話は棚の上にございます。',
        'question': '紙はどこにございますかか。',
        'answer': '机の上にございます。'},]

In [24]:
data = {'lang': 'ja',
        'context': '本はテーブルの上、電話は机の前にあります。',
        'question': '本はどこにありますか。',
        'answer': 'テーブルの上です。'}

place = data['answer'].strip('。')
target = data['question'].strip('?？').split('は')[0]
cop = 'に' + data['context'].strip('。').split('に')[-1]

non_targets = [data['context'].split('、')[1].split('は')[0]]

perfect_matches = [place]
positive_matches = [f"{target}{'は'}{place.lower()}",
                    f"{target}{'が'}{place.lower()}"]
negative_matches = non_targets


print(perfect_matches)
print(positive_matches)
print(negative_matches)



['テーブルの上です']
['本はテーブルの上です', '本がテーブルの上です']
['電話']


## Spatial 2

## Temporal 1

## Temporal 2

## Comparative 1

## Comparative 2

## JA Counters